In [27]:
import re
import sys
import nltk
import string
from math import sqrt, log
from collections import defaultdict
from itertools import chain, product
from nltk import word_tokenize as tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer as stemmer
import numpy as np
from itertools import groupby
from collections import Counter
from nltk.collocations import *
from nltk.stem.porter import PorterStemmer
import pandas as pd
import textfeatures as tf
from sklearn import preprocessing

In [20]:
def compute_average_word_length(sentence):
    return np.mean([len(words) for words in sentence.split()])

def compute_average_sentence_length(sentence):
    sentence = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", sentence)
    return np.mean([len(words) for words in sentence])

def freq_of_words_great_sent_len(sentence):
    result = []
    avg_word_len = compute_average_word_length(sentence)
    # sentence = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", sentence)
    sentence = Counter(sentence.split())
    for key, value in sentence.items():
        if len(key) > avg_word_len:
            result.append(value)
#             print (key, value)
    return sum(result)

def tokenize(sentence):
    return re.split(r"[^0-9A-Za-z\-'_]+", sentence)

def compute_yules_k_for_text(sentence):
    tokens = tokenize(sentence)
    counter = Counter(token.upper() for token in tokens)

    #compute number of word forms in a given sentence/text
    m1 = sum(counter.values())
    m2 = sum([frequency ** 2 for frequency in counter.values()])

    #compute yules k measure and return the value
    yules_k = 10000/((m1 * m1) / (m2 - m1))
    return yules_k


def words_in_sentence(sentence):
    w = [words.strip("0123456789!:,.?()[]{}") for words in sentence.split()]
    return filter(lambda x: len(x) > 0, w)

def compute_yules_i_for_text(sentence):
    dictionary = {}
    stemmer = PorterStemmer()

    for word in words_in_sentence(sentence):
        word = stemmer.stem(word).lower()
        try:
            dictionary[word] += 1
        except:
            dictionary[word] = 1

    m1 = float(len(dictionary))
    m2 = sum([len(list(grouped_values)) * (frequency ** 2) for frequency, grouped_values in groupby(sorted(dictionary.values()))])

    # compute yules i and return the value
    try:
        yules_i = (m1 * m1) / (m2 - m1)
        return yules_i
    except ZeroDivisionError:
        return 0

In [21]:
raw_data = pd.read_csv("scores.csv")
# raw_data = raw_data.sample(frac=0.01).reset_index(drop=True)
raw_data = raw_data.sample(frac=1).reset_index(drop=True)

def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('english')
    content = [w for w in text if w.lower() not in stopwords]
    return content

In [22]:
raw_data["yules_i_measure_abs"] = np.nan
raw_data["yules_i_measure_abs"] = raw_data.apply(lambda x: compute_yules_i_for_text(x['abstract']), axis=1)

raw_data["avg_word_len_abs"] = np.nan
raw_data["avg_word_len_abs"] = raw_data.apply(lambda x: compute_average_word_length(x['abstract']), axis=1)

raw_data["avg_sen_len_abs"] = np.nan
raw_data["avg_sen_len_abs"] = raw_data.apply(lambda x: compute_average_sentence_length(x['abstract']), axis=1)

raw_data["freq_words_gt_sen_len_abs"] = np.nan
raw_data["freq_words_gt_sen_len_abs"] = raw_data.apply(lambda x: freq_of_words_great_sent_len(x['abstract']), axis=1)

raw_data.head()

,abstract,review,semantic_similarity,abstract_clean,review_clean,semantic_similarity_aftercleaning,yules_i_measure_abs,avg_word_len_abs,avg_sen_len_abs,freq_words_gt_sen_len_abs
0,Labeled text classification datasets are typic...,This paper proposes a language independent tex...,[[0.9102851]],labeled text classification dataset typically ...,paper propose language independent text encode...,[[0.8852489]],13.585965,5.086207,116.888889,63
1,We propose and evaluate new techniques for com...,The problem considered in the paper is of comp...,[[0.88059396]],propose evaluate new technique compress speed ...,problem consider paper compress large network ...,[[0.8604121]],31.352227,5.517730,182.800000,65
2,Deep neural networks have become the state-of-...,The authors cast some of the most recent CNN d...,[[0.90264124]],deep neural network state art model numerou...,author cast recent cnn design approximate solu...,[[0.88722855]],20.851064,6.115942,139.285714,124
3,Recurrent Neural Networks (RNNs) continue to s...,UPDATE: Following the author's response I've i...,999999999,recurrent neural networks rnns continue out...,update following author response pron increas...,999999999,32.060606,6.028986,137.714286,53
4,In the pursuit of increasingly intelligent lea...,The paper presents a new policy gradient techn...,[[0.85638505]],pursuit increasingly intelligent learn system ...,paper present new policy gradient technique le...,[[0.7674166]],31.431151,5.743455,142.444444,97


In [23]:
data = raw_data.copy()
data['similarity_score'] = pd.to_numeric(data['semantic_similarity_aftercleaning'].str.replace('[','').str.replace(']',''))
data = data[data['similarity_score'] != 999999999.0].copy()
data['similarity_score'] = data['similarity_score'] * 100

In [25]:
tf.word_count(data,"abstract","word_cnt")
tf.char_count(data,"abstract","char_len")
tf.avg_word_length(data,"abstract","avg_wrd_length")
tf.stopwords_count(data,"abstract","stopwords_cnt")
tf.numerics_count(data,"abstract","num_len")

,abstract,review,semantic_similarity,abstract_clean,review_clean,semantic_similarity_aftercleaning,yules_i_measure_abs,avg_word_len_abs,avg_sen_len_abs,freq_words_gt_sen_len_abs,similarity_score,word_cnt,char_len,avg_wrd_length,stopwords_cnt,num_len
0,Labeled text classification datasets are typic...,This paper proposes a language independent tex...,[[0.9102851]],labeled text classification dataset typically ...,paper propose language independent text encode...,[[0.8852489]],13.585965,5.086207,116.888889,63,88.524890,176,1060,5.086207,69,0
1,We propose and evaluate new techniques for com...,The problem considered in the paper is of comp...,[[0.88059396]],propose evaluate new technique compress speed ...,problem consider paper compress large network ...,[[0.8604121]],31.352227,5.517730,182.800000,65,86.041210,141,918,5.517730,44,0
2,Deep neural networks have become the state-of-...,The authors cast some of the most recent CNN d...,[[0.90264124]],deep neural network state art model numerou...,author cast recent cnn design approximate solu...,[[0.88722855]],20.851064,6.115942,139.285714,124,88.722855,276,1963,6.115942,97,0
4,In the pursuit of increasingly intelligent lea...,The paper presents a new policy gradient techn...,[[0.85638505]],pursuit increasingly intelligent learn system ...,paper present new policy gradient technique le...,[[0.7674166]],31.431151,5.743455,142.444444,97,76.741660,194,1290,5.743455,74,0
5,We exploit a recently derived inversion scheme...,"In summary, the paper is based on a recent wor...",999999999,exploit recently derive inversion scheme arbit...,summary paper base recent work balestriero b...,[[0.7782445]],37.320388,6.070588,119.800000,42,77.824450,86,603,6.070588,28,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2743,Convolutional neural networks (CNNs) are inher...,This paper presents a new convolutional networ...,999999999,convolutional neural network cnns inherently...,paper present new convolutional network archit...,[[0.7938315]],17.384127,6.038760,112.500000,53,79.383150,129,907,6.038760,44,0
2744,Recent DNN pruning algorithms have succeeded i...,This paper casts the pruning optimization prob...,[[0.87137043]],recent dnn prune algorithm succeed reduce numb...,paper cast prune optimization problem nettrim ...,[[0.83231395]],43.742574,5.720588,151.333333,68,83.231395,136,913,5.720588,49,0
2745,Memory Network based models have shown a remar...,The paper proposes to address the quadratic me...,[[0.8944535]],memory network base model remarkable progress ...,paper propose address quadratic memory time r...,[[0.88223743]],25.137931,5.945312,111.500000,71,88.223743,127,899,5.945312,37,0
2746,Reinforcement learning provides a powerful and...,SUMMARY:\r\nThis paper considers the Inverse R...,999999999,reinforcement learn provide powerful general f...,summary paper consider inverse reinforcement...,[[0.7725912]],36.017021,6.049645,200.200000,62,77.259120,129,1005,6.049645,48,0


In [28]:
columns = ['yules_i_measure_abs', 'avg_word_len_abs', 'avg_sen_len_abs', 'freq_words_gt_sen_len_abs', 'similarity_score',
          'word_cnt', 'char_len', 'avg_wrd_length', 'stopwords_cnt', 'num_len']
df = data[columns].copy()
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
#df = pd.DataFrame(x_scaled)
df=pd.DataFrame(x_scaled, columns=df.columns)

In [29]:
training_set = data.sample(frac=0.8, random_state = 1)
test_set = data.loc[~data.index.isin(training_set.index)]
test_set.head()

training_set = df.sample(frac=0.8, random_state = 1)
test_set = df.loc[~df.index.isin(training_set.index)]
test_set.head()

,yules_i_measure_abs,avg_word_len_abs,avg_sen_len_abs,freq_words_gt_sen_len_abs,similarity_score,word_cnt,char_len,avg_wrd_length,stopwords_cnt,num_len
14,0.152054,0.339162,0.446057,0.315436,0.674371,0.353116,0.368834,0.339162,0.343284,0.0
15,0.072591,0.227040,0.351423,0.402685,0.469683,0.344214,0.381282,0.227040,0.365672,0.0
18,0.182241,0.270005,0.447777,0.543624,0.882892,0.528190,0.528354,0.270005,0.425373,0.0
20,0.679690,0.577372,0.565251,0.234899,0.737347,0.216617,0.258645,0.577372,0.186567,0.0
25,0.482065,0.409590,0.358191,0.362416,0.827574,0.290801,0.315814,0.409590,0.261194,0.0


In [30]:
data_columns = ['yules_i_measure_abs', 'avg_word_len_abs',
       'avg_sen_len_abs', 'freq_words_gt_sen_len_abs']

columns = ['yules_i_measure_abs', 'avg_word_len_abs', 'avg_sen_len_abs', 'freq_words_gt_sen_len_abs',
          'word_cnt', 'char_len', 'avg_wrd_length', 'stopwords_cnt', 'num_len']


# training_data = training_set.as_matrix(columns = data_columns)
training_data = training_set[data_columns].to_numpy()# .as_matrix(columns = data_columns)
nan_locs = np.isnan(training_data)
training_data[nan_locs] = 0

training_target = training_set['similarity_score'].values
nan_locs = np.isnan(training_target)
training_target[nan_locs] = 0

test_data = test_set[data_columns].to_numpy()# .as_matrix(columns = data_columns)
nan_locs = np.isnan(test_data)
test_data[nan_locs] = 0

test_target = test_set['similarity_score'].values
nan_locs = np.isnan(test_target)
test_target[nan_locs] = 0

In [31]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor()
reg.fit(training_data, training_target)

print('R squared: ' + str(reg.score(test_data, test_target)))

R squared: 0.12011333679347957


In [32]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(training_data, training_target)

print('R squared: ' + str(svr.score(test_data, test_target)))

R squared: 0.1448915398016214


In [33]:
from sklearn.kernel_ridge import KernelRidge
kr = KernelRidge()
kr.fit(training_data, training_target)

print('R squared: ' + str(kr.score(test_data, test_target)))

R squared: -0.7980933809428177


In [34]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dt.fit(training_data, training_target)

print('R squared: ' + str(dt.score(test_data, test_target)))

R squared: -0.07324147924227065


In [46]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 2000)
rf.fit(training_data, training_target)

print('R squared: ' + str(rf.score(test_data, test_target)))

R squared: 0.08143122239111589


In [36]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor()
gb.fit(training_data, training_target)

print('R squared: ' + str(gb.score(test_data, test_target)))

R squared: 0.1564466776096709


In [37]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

pr = Pipeline([('poly', PolynomialFeatures(degree = 3)),
              ('linear', LinearRegression(fit_intercept = False))])

pr.fit(training_data, training_target)

print('R squared: ' + str(pr.score(test_data, test_target)))

R squared: 0.13190546464869635


In [38]:
from sklearn.linear_model import Lasso
lm = Lasso()
lm.fit(training_data, training_target)

print('R squared: ' + str(lm.score(test_data, test_target)))

R squared: -0.0008193996197520814


In [39]:
from sklearn.linear_model import ElasticNet
en = ElasticNet()
en.fit(training_data, training_target)

print('R squared: ' + str(en.score(test_data, test_target)))

R squared: -0.0008193996197520814


In [40]:
from sklearn.linear_model import LassoLars
ll = LassoLars()
ll.fit(training_data, training_target)

print('R squared: ' + str(ll.score(test_data, test_target)))

R squared: -0.0008193996197520814


In [41]:
from sklearn.linear_model import BayesianRidge
br = BayesianRidge()
br.fit(training_data, training_target)

print('R squared: ' + str(br.score(test_data, test_target)))

R squared: 0.11388019430554819


In [42]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor()
sgd.fit(training_data, training_target)

print('R squared: ' + str(sgd.score(test_data, test_target)))

R squared: 0.027637723225301447
